In [1]:
# 分词
# 词语 -> id
#   matrix -> [|V|, embed_size]
#   词语A -> id(5)
#   词表

# label -> id

import sys
import os
import jieba # pip install jieba

# input files
train_file = '.\\cnews_data\\cnews.train.txt'
val_file = '.\\cnews_data\\cnews.val.txt'
test_file = '.\\cnews_data\\cnews.test.txt'

# output files
seg_train_file = '.\\cnews_data\\cnews.train.seg.txt'
seg_val_file = '.\\cnews_data\\cnews.val.seg.txt'
seg_test_file = '.\\cnews_data\\cnews.test.seg.txt'

vocab_file = '.\\cnews_data\\cnews.vocab.txt'#词表文件，词表到id的映射
category_file = '.\\cnews_data\\cnews.category.txt'#label到id的映射

In [2]:
with open(val_file,encoding='utf-8') as f:
    lines = f.readlines()

label, content = lines[0].strip('\r\n').split('\t')#strip去掉分隔符，split用table键进行分割
word_iter = jieba.cut(content)#结巴分词api自动拆分句子成词组

print(content)
print('|'.join(word_iter))#把content里列表的内容用|合并起来

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\TH\AppData\Local\Temp\jieba.cache


黄蜂vs湖人首发：科比带伤战保罗 加索尔救赎之战 新浪体育讯北京时间4月27日，NBA季后赛首轮洛杉矶湖人主场迎战新奥尔良黄蜂，此前的比赛中，双方战成2-2平，因此本场比赛对于两支球队来说都非常重要，赛前双方也公布了首发阵容：湖人队：费舍尔、科比、阿泰斯特、加索尔、拜纳姆黄蜂队：保罗、贝里内利、阿里扎、兰德里、奥卡福[新浪NBA官方微博][新浪NBA湖人新闻动态微博][新浪NBA专题][黄蜂vs湖人图文直播室](新浪体育)


Loading model cost 0.996 seconds.
Prefix dict has been built succesfully.


黄蜂|vs|湖人|首发|：|科比|带伤|战|保罗| |加索尔|救赎|之战| |新浪|体育讯|北京|时间|4|月|27|日|，|NBA|季后赛|首轮|洛杉矶|湖人|主场|迎战|新奥尔良|黄蜂|，|此前|的|比赛|中|，|双方|战成|2|-|2|平|，|因此|本场|比赛|对于|两支|球队|来说|都|非常|重要|，|赛前|双方|也|公布|了|首发|阵容|：|湖人队|：|费舍尔|、|科比|、|阿泰斯特|、|加索尔|、|拜纳姆|黄蜂队|：|保罗|、|贝里|内利|、|阿里|扎|、|兰德|里|、|奥卡福|[|新浪|NBA|官方|微博|]|[|新浪|NBA|湖人|新闻动态|微博|]|[|新浪|NBA|专题|]|[|黄蜂|vs|湖人|图文|直播室|]|(|新浪|体育|)


In [12]:
def generate_seg_file(input_file, output_seg_file):#生成分词后的文件，输入文件，输出文件
    """Segment the sentences in each line in input_file"""
    with open(input_file,'r',encoding='utf-8') as f:
        lines = f.readlines()
    with open(output_seg_file,'w',encoding='utf-8') as f:
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            word_iter = jieba.cut(content)
            word_content = ''
            for word in word_iter:
                word = word.strip(' ')#jieba会把所有的空格也拆分成词组，这时会把所有的空格strip成空字符串
                if word != '':
                    word_content += word + ' '#每个词用空格进行分割
            out_line = '%s\t%s\n' % (label, word_content.strip(' '))
            f.write(out_line)

generate_seg_file(train_file, seg_train_file)
generate_seg_file(val_file, seg_val_file)
generate_seg_file(test_file, seg_test_file)

In [15]:
#构建词表
def generate_vocab_file(input_seg_file, output_vocab_file):
    with open(input_seg_file,'r',encoding='utf-8') as f:
        lines = f.readlines()
    word_dict = {}#用字典保存每个词语的频率信息
    for line in lines:
        label, content = line.strip('\r\n').split('\t')#上一个文件用空格区分，现在用空格拆分
        for word in content.split():
            word_dict.setdefault(word, 0)
            word_dict[word] += 1#统计频率
    
    sorted_word_dict = sorted( #对词频逆排序                  sorted的格式：[(word, frequency), ..., ()]
        word_dict.items(), key = lambda d:d[1], reverse=True)#把字典变成列表，列表的每一个元素都是词语的频次，按列表的第二个值排序d[1]
    with open(output_vocab_file, 'w',encoding='utf-8') as f:
        f.write('<UNK>\t10000000\n')#如果找不到就返回UNK这个词，UNK词频为1000000，比如测试集出现的词训练集没有，所以就用UNK代替这个没出现的词
        for item in sorted_word_dict:
            f.write('%s\t%d\n' % (item[0], item[1]))

generate_vocab_file(seg_train_file, vocab_file)#只在训练集统计词表，因为上线之前不知道会有什么其他词

In [17]:
#生成label信息
def generate_category_dict(input_file, category_file):
    with open(input_file, 'r',encoding='utf-8') as f:
        lines = f.readlines()
    category_dict = {}
    for line in lines:
        label, content = line.strip('\r\n').split('\t')
        category_dict.setdefault(label, 0)
        category_dict[label] += 1；
    category_number = len(category_dict)
    with open(category_file, 'w',encoding='utf-8') as f:
        for category in category_dict:
            line = '%s\n' % category
            print('%s\t%d' % (category, category_dict[category]))
            f.write(line)
            
generate_category_dict(train_file, category_file)

体育	5000
娱乐	5000
家居	5000
房产	5000
教育	5000
时尚	5000
时政	5000
游戏	5000
科技	5000
财经	5000
